### Require Dependencies

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LSTM, Dense
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np 

### Preprocess Data

In [2]:
df = pd.read_csv('data.csv', header=0)
df.head()


,date,onpromotion,storeNbr,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,...,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD,totalSales
0,2013-01-01,0,1,0,0,0,0,0,0.0,0,...,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0
1,2013-01-01,0,2,0,0,0,0,0,0.0,0,...,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0
2,2013-01-01,0,3,0,0,0,0,0,0.0,0,...,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0
3,2013-01-01,0,4,0,0,0,0,0,0.0,0,...,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0
4,2013-01-01,0,5,0,0,0,0,0,0.0,0,...,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0


In [3]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        dataX.append(dataset[i:(i + look_back), :])
        dataY.append(dataset[i + look_back, 3:-1])
    return np.array(dataX), np.array(dataY)


### Define Architecture

In [12]:
look_back = 15
num_categories = len(df.columns) - 4  # Excluding date, storeNbr, onpromotion, and totalSales

model = Sequential()
model.add(LSTM(256, input_shape=(look_back, num_categories + 2)))  # +3 for storeNbr, onpromotion, and totalSales
model.add(Dense(32))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 256)               299008    
                                                                 
 dense_1 (Dense)             (None, 32)                8224      
                                                                 
Total params: 307,232
Trainable params: 307,232
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [13]:
# Prepare the dataset
df = pd.read_csv('data.csv', header=0)
df['date'] = pd.to_datetime(df['date'])
df = df[df['storeNbr'] == 1] # start with only the first store
data = df.sort_values(by=['date']).drop(columns=['date','storeNbr']).values


# Create input-output pairs
X, y = create_dataset(data, look_back)

# Standardize the data
scaler = StandardScaler()
data = scaler.fit_transform(data)


# Split the data into training and testing sets
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2, random_state=42)


# history = model.fit(trainX, trainY, epochs=100, batch_size=32, verbose=2, validation_data=(testX, testY))


Epoch 1/100


ValueError: in user code:

    File "c:\Users\rykun\OneDrive - Michigan State University\Machine Learning\CSE404-Group11\env\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\rykun\OneDrive - Michigan State University\Machine Learning\CSE404-Group11\env\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\rykun\OneDrive - Michigan State University\Machine Learning\CSE404-Group11\env\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\rykun\OneDrive - Michigan State University\Machine Learning\CSE404-Group11\env\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\rykun\OneDrive - Michigan State University\Mac

In [6]:
# Needs a TON of epochs
for i in range(20):
    # Prepare the dataset
    df = pd.read_csv('data.csv', header=0)
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['storeNbr'] == i+1] # start with only the first store
    data = df.sort_values(by=['date']).drop(columns=['date','storeNbr']).values


    # Create input-output pairs
    X, y = create_dataset(data, look_back)

    # Split the data into training and testing sets
    trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2, random_state=42)

    history = model.fit(trainX, trainY, epochs=100, batch_size=32, verbose=2, validation_data=(testX, testY))

    # Get the training loss and accuracy from the history object
    training_loss = history.history['loss']
    val_loss = history.history['val_loss']


    # Plot the training loss and accuracy over the epochs
    epochs = range(1, len(training_loss) + 1)

    plt.plot(epochs, training_loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node 'while/MatMul' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\rykun\OneDrive - Michigan State University\Machine Learning\CSE404-Group11\env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\rykun\OneDrive - Michigan State University\Machine Learning\CSE404-Group11\env\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\rykun\OneDrive - Michigan State University\Machine Learning\CSE404-Group11\env\lib\site-packages\i

### Test Performance

In [10]:

# Test the model and make predictions
testPredictions = model.predict(testX)

# Calculate the Mean Squared Error (MSE) on the test data
test_mse = mean_squared_error(testY, testPredictions)
print("Mean Squared Error: ", test_mse)

test_mae = mean_absolute_error(testY, testPredictions)
test_r2 = r2_score(testY, testPredictions)

print("Mean Absolute Error:", test_mae)
print("R-squared score:", test_r2)
"""
Mean Squared Error:  66174.43491272551
Mean Absolute Error: 81.56172771348042
R-squared score: 0.13775325212931822
oof
"""
"""
Mean Squared Error:  68592.14438589144
Mean Absolute Error: 84.0454415952677
R-squared score: 0.14306700673409384
"""



23/23 [==============================] - 0s 9ms/step
Mean Squared Error:  68592.14438589144
Mean Absolute Error: 84.0454415952677
R-squared score: 0.14306700673409384


'\nMean Squared Error:  68592.14438589144\nMean Absolute Error: 84.0454415952677\nR-squared score: 0.14306700673409384\n'

In [ ]:
x = np.array([testX[-1]]) # predict last point
model.predict(x)


"""
array([[-6.1686362e-05,  4.6692055e-01,  3.8359444e+01, -2.6546391e-05,
         1.8567648e+02,  1.6154854e+00,  4.1970105e+02,  3.5754965e+02,
         3.5898205e+01,  1.7267962e+01,  9.8091194e+01, -2.0850977e+02,
         5.2151585e+00, -4.8492447e-02,  1.9163357e+00,  4.5457201e+00,
        -1.4824487e-02,  3.1117912e+01,  7.9827392e-01,  2.3482652e+00,
         1.7561276e+00,  2.7935734e+01,  9.0150878e-02,  8.9242462e+01,
         5.5166054e+01,  2.0977840e-01,  1.1819119e+00,  3.2852470e+01,
         3.1793232e+00,  4.8103748e+01,  1.4592125e-01,  3.9945490e+00]
"""


### Notes

- Model takes way too long to train
- Most likely need to normalize
- Considerations for null data?
- Architecture for store specific model
    - Encapsulate BST
- Grid search hyperparams
    - window size
    - optimizer